In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging

from depsurf import OUTPUT_PATH, VERSION_LAST, Funcs, InlineType, Version, SymbolInfo


def count_inline(v: Version):
    funcs = Funcs.from_jsonl(v.funcs_path)

    results = {t: Funcs() for t in InlineType}
    symtab = SymbolInfo.from_dump(v.symtab_path)
    symtab_funcs = {n.split(r".", 1)[0] for n in symtab.funcs["name"]}

    for func in funcs.data:
        has_symtab = func.name in symtab_funcs

        if func.inline_actual:
            if not func.has_inline_caller:
                logging.debug(
                    f"{func.name} at {func.loc} is inline but has no inline caller. "
                    f"maybe it is declared w/ __attribute__((always_inline))"
                )

            if func.has_func_caller and not has_symtab:
                logging.warning(
                    f"{func.name} at {func.loc} has func caller but no symtab entry. "
                )

            if has_symtab:
                results[InlineType.PARTIAL].add(func)
            else:
                results[InlineType.FULL].add(func)
        else:
            if func.has_inline_caller:
                logging.warning(
                    f"{func.name} at {func.loc} not inlined but has inline caller. "
                )
            results[InlineType.NOT].add(func)

    for t, f in results.items():
        f.save_result(OUTPUT_PATH / "inline" / v.name / f"{t.name.lower()}.txt")

    return {t: f.num_funcs for t, f in results.items()}


# count_inline(VERSION_LAST)

In [3]:
from depsurf import VersionGroup
from depsurf.output import save_df

df = VersionGroup.apply(count_inline)

save_df(df, "inline")

[    utils.py:160] INFO: NumExpr defaulting to 8 threads.
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.pkl
[    funcs.py:39 ] INFO: Saved 65697 functions to /Users/szhong/Downloads/bpf-study/output/inline/4.4.0-21-generic-amd64/full.txt
[    funcs.py:39 ] INFO: Saved 31759 functions to /Users/szhong/Downloads/bpf-study/output/inline/4.4.0-21-generic-amd64/not.txt
[    funcs.py:39 ] INFO: Saved 9766 functions to /Users/szhong/Downloads/bpf-study/output/inline/4.4.0-21-generic-amd64/partial.txt
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.pkl
[    funcs.py:39 ] INFO: Saved 69785 functions to /Users/szhong/Downloads/bpf-study/output/inline/4.8.0-22-generic-amd64/full.txt
[    funcs.py:39 ] INFO: Saved 34555 functions to /Users/szhong/Downloads/bpf-study/output/inline/4.8.0-22-generic-amd64/not.txt
[    funcs.py:39 ] INFO: Saved 10622 functions to /Users

Fully inlined  Not inlined  Partially inlined
reg    4.4.0-21-generic-amd64             65697        31759               9766
       4.8.0-22-generic-amd64             69785        34555              10622
       4.10.0-19-generic-amd64            72800        35405              10999
       4.13.0-16-generic-amd64            77715        36382              10199
       4.15.0-20-generic-amd64            79521        37778              11331
       4.18.0-10-generic-amd64            85915        40527              11200
       5.0.0-13-generic-amd64             89330        41284              11262
       5.3.0-18-generic-amd64             95153        42496              12454
       5.4.0-26-generic-amd64             97476        43279              12823
       5.8.0-25-generic-amd64             96778        50593              15047
       5.11.0-16-generic-amd64           102577        52788              15385
       5.13.0-19-generic-amd64           106671        51945              15942
       5.15.0-25-generic-amd64           108669        52864              16130
       5.19.0-21-generic-amd64           113455        54874              13852
       6.2.0-20-generic-amd64            117061        56038              12451
       6.5.0-9-generic-amd64             123204        58340              12880
       6.8.0-22-generic-amd64            130551        60769              13802
arch   5.4.0-26-generic-arm64            122416        48702              22203
       5.4.0-26-generic-armhf             92867        44394              10522
       5.4.0-26-generic-ppc64el           87083        39417               9561
flavor 5.4.0-26-lowlatency-amd64          95800        43318              12049
       5.4.0-1009-aws-amd64               95203        41918              12439
       5.4.0-1009-gcp-amd64               98401        43405              12806
       5.4.0-1010-azure-amd64             95424        40955              11305